In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = "/kaggle/input/tabular-playground-series-jan-2021/train.csv"
test_path = "/kaggle/input/tabular-playground-series-jan-2021/test.csv"
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
col = df_train.columns
df_test.head()

In [ ]:
#build simple linear regression and see accuracy
from sklearn import linear_model
X = df_train[col[1:15]]
y = df_train['target']

linear = linear_model.LinearRegression(fit_intercept = True)

linear.fit(X, y)

In [ ]:
#linear model feature importance
from matplotlib import pyplot
coef = linear.coef_

stdev = []
for i in range(len(df_train.columns)):
    if i != 0 and i != 15:
        stdev.append(df_train[col[i]].std())
        
df_importance = []
for i,v in enumerate(coef):
    importance = coef[i] * stdev[i]
    df_importance.append(importance)
    print('Feature: %0d, Score: %.5f' % (i,importance))
df_importance = pd.DataFrame(df_importance)
df_importance
    
pyplot.bar([x for x in range(len(df_importance))], abs(df_importance[0]))

In [ ]:
X_normalized = (X-X.mean())/X.std()

linear.fit(X_normalized, y)

from sklearn.metrics import confusion_matrix
y_pred = linear.predict(df_test[col[1:15]])
pred = pd.DataFrame(y_pred)
submission = pd.concat([df_test['id'], pred], axis = 1)
submission.columns = ['id', 'target']
submission.to_csv("normalized.csv", index = False)
#RMSE Error: 0.73333

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = linear.predict(df_test[col[1:15]])
pred = pd.DataFrame(y_pred)
submission = pd.concat([df_test['id'], pred], axis = 1)
submission.columns = ['id', 'target']
submission.to_csv("first_submission.csv", index = False)
#RMSE Error: 0.72782

In [ ]:
from sklearn.model_selection import train_test_split
X_train_train, X_train_test = train_test_split(df_train, test_size=0.20, random_state=1)
X_train_test_values = pd.DataFrame(X_train_test[col[1:15]])
X_train_test_target = pd.DataFrame(X_train_test['target'])

X_train_train_values = pd.DataFrame(X_train_train[col[1:15]])
X_train_train_target = pd.DataFrame(X_train_train['target'])

In [ ]:
#optimize linear regression
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(5, 3, figsize = (20,25))
#x = df_train[col[4]]
y = df_train["target"]

index = 1
for i in range(5):
    for j in range(3):
        x = df_train[col[index]]
        sns.scatterplot(ax = axes[i, j], x = x[0:20000], y = y[0:20000])
        axes[i][j].set_title(f"Plot for cont{index}")
        index += 1



In [ ]:
fig2, axes2 = plt.subplots(5, 3, figsize = (20,25))

index = 1
for i in range(5):
    for j in range(3):
        sns.distplot(df_train[col[index]], ax = axes2[i][j])
        axes[i][j].set_title(f"Plot for cont{index}")
        index += 1


In [ ]:
log_transformed = []
for i in range(len(df_train.columns)):
    log_transformed.append(np.log1p(df_train[col[i]]))
log_transformed = pd.DataFrame(log_transformed).T

fig2, axes2 = plt.subplots(5, 3, figsize = (20,25))

index = 1
for i in range(5):
    for j in range(3):
        sns.distplot(log_transformed[col[index]], ax = axes2[i][j])
        axes[i][j].set_title(f"Plot for cont{index}")
        index += 1
        


In [ ]:
from sklearn.metrics import mean_squared_error
linear_pred = linear.predict(X_train_test_values)
score_linear = mean_squared_error(X_train_test_target, linear_pred,squared=False)
print(f'{score_linear:0.5f}')

In [ ]:
#Tree models
from sklearn.tree import DecisionTreeRegressor
d_tree = DecisionTreeRegressor(random_state=0)
d_tree.fit(X_train_train_values, X_train_train_target)
y_dtree = d_tree.predict(X_train_test_values)

In [ ]:
score_dtree = mean_squared_error(X_train_test_target, y_dtree, squared=False)
print(f'{score_dtree:0.5f}')

In [ ]:
y_pred = pd.DataFrame(y_dtree)
plt.figure(figsize = (10,5))
sns.scatterplot(X_train_test_target['target'], y_pred[0])

In [ ]:
#Random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

estimators = [30, 50, 60, 70]
max_depth = [None, 2, 3]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

accuracy = []
for i in estimators:
    for j in max_depth:
        for a in min_samples_split:
            for b in min_samples_leaf:
                for c in bootstrap:
                    rf_model = RandomForestRegressor(n_estimators=i, n_jobs=-1, max_depth = j, min_samples_split = a, min_samples_leaf = b, bootstrap = c)
                    rf_model.fit(X_train_train_values, X_train_train_target)

                    y_rf = rf_model.predict(X_train_test_values)
                    score_rf = mean_squared_error(X_train_test_target, y_rf, squared=False)
                    accuracy.append([i, j, a, b, c, score_rf])
                    print(i, j, a, b, c, score_rf)
results = pd.DataFrame(data = accuracy, columns = ["estimator", "max_depth", "samples_split", "samples_leaf", "bootstrap", "score"])
results

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 0, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 2, verbose=10, random_state=42, n_jobs = -1)
rf_random.fit(X_train_train_values, X_train_train_target)

In [ ]:
 rf_random.best_params_

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model = tf.keras.Sequential( … ) # define your model normally
    model.compile( … )

# train model normally
model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

In [ ]:
results.sort_values(by = ['score'], ascending = True)
#best combo = 70, NaN for estimator and max_depth

In [ ]:
X = df_train[col[1:15]]
y = df_train['target']

rf_model = RandomForestRegressor(n_estimators=70, n_jobs=-1, max_depth = None)

rf_model.fit(X, y)

rf_y_pred = rf_model.predict(df_test[col[1:15]])

pred = pd.DataFrame(rf_y_pred)
submission = pd.concat([df_test['id'], pred], axis = 1)
submission.columns = ['id', 'target']
submission.to_csv("rf_model.csv", index = False)

In [ ]:
y_pred_rf = pd.DataFrame(y_rf)
plt.figure(figsize = (10,5))
sns.scatterplot(X_train_test_target['target'], y_pred_rf[0])
plt.plot([4, 10], [4,10])

In [ ]:
importance_rf = rf_model.feature_importances_ 

importance_df = []
# summarize feature importance
for i,v in enumerate(importance_rf):
    print('Feature: %0d, Score: %.5f' % (i+1,v))
    importance_df.append(v)
# plot feature importance
plt.ylim([0.0675, 0.08])
pyplot.bar([x for x in range(len(importance_rf))], abs(importance_rf))
pyplot.show()
importance_df = pd.DataFrame( data = importance_df).sort_values(by = [0], ascending = False)

In [ ]:
#Important features: 0- 3, 6, 9, 12
important_features_rf = pd.DataFrame([df_train[col[2]], df_train[col[3]], df_train[col[4]], df_train[col[7]], df_train[col[10]], df_train[col[13]]]).T
important_features_rf.columns = ["cont2", "cont3", "cont4", "cont7", "cont10", "cont13" ]
important_features_rf

In [ ]:
importance_rf = pd.DataFrame(importance_rf)
sns.set_palette("husl")
sns.heatmap(df_train[col[1:16]].corr(), cmap = "PiYG")

In [ ]:
pyplot.bar([x for x in range(len(df_importance))], abs(df_importance[0]))

In [ ]:
#feature creation
important_features_rf_squared = important_features_rf ** 2
important_features_rf_squared.columns = ["cont2_sq", "cont3_sq", "cont4_sq", "cont7_sq", "cont10_sq", "cont13_sq" ]
mult_3_4 = important_features_rf["cont3"] * important_features_rf["cont4"]
mult_3_4.columns = ["3*4"]
mult_2_3 = important_features_rf["cont3"] * important_features_rf["cont2"]
mult_2_3.columns = ["2*3"]
mult_2_4 = important_features_rf["cont2"] * important_features_rf["cont4"]
mult_2_4.columns = ["2*4"]
new_important_features = pd.concat([important_features_rf_squared, mult_3_4, mult_2_3, mult_2_4], axis = 1)
new_important_features.columns = ["cont2_sq", "cont3_sq", "cont4_sq", "cont7_sq", "cont10_sq", "cont13_sq", "3*4", "2*3", "2*4"]
new_important_features

In [ ]:
pd.DataFrame(new_important_features)

In [ ]:
rf_train = pd.concat([df_train, new_important_features], axis = 1)
rf_train_values = rf_train.drop(['id', 'target'], axis = 1)
col_train = rf_train_values.columns
#normal distribution
log_transformed = []
for i in range(len(rf_train_values.columns)):
    log_transformed.append(np.log1p(rf_train_values[col_train[i]]))
log_transformed_rf_train_values = pd.DataFrame(log_transformed).T



rf_train_target = rf_train['target']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=70, n_jobs=-1, max_depth = None)

rf_model.fit(log_transformed_rf_train_values, rf_train_target)



In [ ]:
#prepare validation data
validation_xgb = pd.DataFrame(X_train_test[col[0:15]])
X_train_test_target = pd.DataFrame(X_train_test['target'])

#get test data prepared
validation = pd.DataFrame([validation_xgb[col[2]], validation_xgb[col[3]], validation_xgb[col[4]], validation_xgb[col[7]], validation_xgb[col[10]], validation_xgb[col[12]]]).T
validation.columns = ["cont2", "cont3", "cont4", "cont7", "cont10", "cont13" ]

important_validation_squared = validation ** 2
important_validation_squared.columns = ["cont2_sq", "cont3_sq", "cont4_sq", "cont7_sq", "cont10_sq", "cont13_sq" ]
mult_3_4_validation = validation["cont3"] * validation["cont4"]
mult_3_4_validation.columns = ["3*4"]
mult_2_3_validation = validation["cont3"] * validation["cont2"]
mult_2_3_validation.columns = ["2*3"]
mult_2_4_validation = validation["cont2"] * validation["cont4"]
mult_2_4_validation.columns = ["2*4"]
new_important_features_validation = pd.concat([important_validation_squared, mult_3_4_validation, mult_2_3_validation, mult_2_4_validation], axis = 1)
new_important_features_validation.columns = ["cont2_sq", "cont3_sq", "cont4_sq", "cont7_sq", "cont10_sq", "cont13_sq", "3*4", "2*3", "2*4"]
new_important_features_validation


xgb_val = pd.concat([validation_xgb, new_important_features_validation], axis = 1)
xgb_val = xgb_val.drop(['id'], axis = 1)
col_val = xgb_val.columns

#normal distribution
log_transformed = []
for i in range(len(xgb_val.columns)):
    log_transformed.append(np.log1p(xgb_val[col_test[i]]))
validation_dataset = pd.DataFrame(log_transformed).T


In [ ]:
#get test data prepared
test_important = pd.DataFrame([df_test[col[2]], df_test[col[3]], df_test[col[4]], df_test[col[7]], df_test[col[10]], df_test[col[12]]]).T
test_important.columns = ["cont2", "cont3", "cont4", "cont7", "cont10", "cont13" ]

important_features_test_squared = test_important ** 2
important_features_test_squared.columns = ["cont2_sq", "cont3_sq", "cont4_sq", "cont7_sq", "cont10_sq", "cont13_sq" ]
mult_3_4_test = test_important["cont3"] * test_important["cont4"]
mult_3_4_test.columns = ["3*4"]
mult_2_3_test = test_important["cont3"] * test_important["cont2"]
mult_2_3_test.columns = ["2*3"]
mult_2_4_test = test_important["cont2"] * test_important["cont4"]
mult_2_4_test.columns = ["2*4"]
new_important_features_test = pd.concat([important_features_test_squared, mult_3_4_test, mult_2_3_test, mult_2_4_test], axis = 1)
new_important_features_test.columns = ["cont2_sq", "cont3_sq", "cont4_sq", "cont7_sq", "cont10_sq", "cont13_sq", "3*4", "2*3", "2*4"]
new_important_features_test


rf_test = pd.concat([df_test, new_important_features_test], axis = 1)
rf_test = rf_test.drop(['id'], axis = 1)
col_test = rf_test.columns

#normal distribution
log_transformed = []
for i in range(len(rf_test.columns)):
    log_transformed.append(np.log1p(rf_test[col_test[i]]))
prediction_dataset = pd.DataFrame(log_transformed).T
prediction_dataset

In [ ]:
pred = rf_model.predict(prediction_dataset)

pred = pd.DataFrame(pred)
submission = pd.concat([df_test['id'], pred], axis = 1)
submission.columns = ['id', 'target']
submission.to_csv("rf_model2.csv", index = False)


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
model_XGB = XGBRegressor(                 
                colsample_bytree=0.5,
                 alpha=0.01563,
                 #gamma=0.0,
                 learning_rate=0.01,
                 max_depth=15,
                 min_child_weight=257,
                 n_estimators=1000,                                                                  
                 #reg_alpha=0.9,
                 reg_lambda=0.003,
                 subsample=0.7,
                 random_state=2020,
                 metric_period=100,
                 silent=1,
                n_jobs = -1)
print('ready to train')
model_XGB.fit(log_transformed_rf_train_values, rf_train_target, early_stopping_rounds=6, eval_set=[(validation_dataset, X_train_test_target)], verbose=1)
print("predicting")
y_xgb = model_XGB.predict(validation_dataset)

score_xgb = mean_squared_error(X_train_test_target, y_xgb, squared=False)
print(f'{score_xgb:0.5f}')

In [ ]:
print("predicting")
y_xgb = model_XGB.predict(validation_dataset)

score_xgb = mean_squared_error(X_train_test_target, y_xgb, squared=False)
print(f'{score_xgb:0.5f}')

In [ ]:
#model_XGB.fit(log_transformed_rf_train_values, rf_train_target)
xgb_pred = model_XGB.predict(prediction_dataset)

pred = pd.DataFrame(xgb_pred)
submission = pd.concat([df_test['id'], pred], axis = 1)
submission.columns = ['id', 'target']
submission.to_csv("xgb2.csv", index = False)

In [ ]:
submission